<a href="https://colab.research.google.com/github/TMUITLab/EAFR/blob/master/EA15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd '/content'
!git clone https://github.com/1049451037/GCN-Align
!git clone https://github.com/MaoXinn/RREA

/content
Cloning into 'GCN-Align'...
remote: Enumerating objects: 53, done.
remote: Total 53 (delta 0), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (53/53), done.
Cloning into 'RREA'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 32 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [2]:
import warnings
warnings.filterwarnings('ignore')

from importlib.machinery import SourceFileLoader
from google.colab import files

layer = SourceFileLoader("layer", "/content/RREA/CIKM/layer.py").load_module()
utils = SourceFileLoader("utils", "/content/RREA/CIKM/utils.py").load_module()
CSLS = SourceFileLoader("CSLS", "/content/RREA/CIKM/CSLS.py").load_module()
import tensorflow as tf
import os
import time
import random
import keras
import pickle
from scipy.sparse import hstack
import math
from tqdm import *
import numpy as np
from utils import *
from CSLS import *
import tensorflow as tf
import keras.backend as K
from keras.layers import *
from layer import NR_GraphAttention

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [3]:
lang = 'zh'
main_radj,train_pair,dev_pair,adj_matrix,r_index,r_val,adj_features,rel_features,nadj,char_size = load_data('/content/GCN-Align/data/%s_en/'%lang,train_ratio=0.30)
train_pair_main=train_pair
adj_matrix = np.stack(adj_matrix.nonzero(),axis = 1)
main_rel_matrix,rel_val = np.stack(rel_features.nonzero(),axis = 1),rel_features.data
ent_matrix,ent_val = np.stack(adj_features.nonzero(),axis = 1),adj_features.data

38960 3024


In [4]:
rel_size = rel_features.shape[1] 

train_pair_dic = dict()
for i in range(len(train_pair)):
    train_pair_dic[train_pair[i][0]] = train_pair[i][1]

train_pair_dic = dict()
pair_dic = dict()
pairs = np.concatenate((dev_pair , train_pair),axis=0)
for i in range(len(train_pair)):
    train_pair_dic[train_pair[i][0]] = train_pair[i][1]
for i in range(len(pairs)):
    pair_dic[pairs[i][0]] = pairs[i][1]


main_rel_matrix1 = main_rel_matrix.copy()
for i in range(len(main_rel_matrix1)):
  main_rel_matrix1[i,1] = (main_rel_matrix1[i,1] + rel_size//2) % rel_size

main_radj1 = main_radj.copy()
for i in range(len(main_radj1)):
  main_radj1[i,2] = (main_radj1[i,2] + rel_size//2) % rel_size
single_triple_set = set()
degree = np.zeros([adj_features.shape[0],3])
for triple in main_radj:
    single_triple_set.add((triple[0],triple[1]))
    if triple[2] < rel_size //2:
        degree[triple[0],0] += 1
        degree[triple[1],0] += 1
        degree[triple[0],2] += 1
        degree[triple[1],1] += 1

dev_pair_degrees = np.zeros([2,len(dev_pair),3])
for i in range(len(dev_pair)):
    dev_pair_degrees[0,i,0] = degree[dev_pair[i][0],0]
    dev_pair_degrees[0,i,1] = degree[dev_pair[i][0],1]
    dev_pair_degrees[0,i,2] = degree[dev_pair[i][0],2]
    dev_pair_degrees[1,i,0] = degree[dev_pair[i][1],0]
    dev_pair_degrees[1,i,1] = degree[dev_pair[i][1],1]
    dev_pair_degrees[1,i,2] = degree[dev_pair[i][1],2]

train_pair_degrees = np.zeros([2,len(train_pair),3])
for i in range(len(train_pair)):
    train_pair_degrees[0,i,0] = degree[train_pair[i][0],0]
    train_pair_degrees[0,i,1] = degree[train_pair[i][0],1]
    train_pair_degrees[0,i,2] = degree[train_pair[i][0],2]
    train_pair_degrees[1,i,0] = degree[train_pair[i][1],0]
    train_pair_degrees[1,i,1] = degree[train_pair[i][1],1]
    train_pair_degrees[1,i,2] = degree[train_pair[i][1],2]

ent_size = adj_features.shape[0]
depth = 3
deg = np.zeros([ent_size])
deg_in = np.zeros([ent_size])
deg_out = np.zeros([ent_size])
bdeg = np.zeros([ent_size])


single_graph = dict()
for h,t,r in main_radj:
    deg[h] += 1
    deg[t] += 1
    deg_in[t] += 1
    deg_out[h] += 1
    if(h not in single_graph):
        single_graph[h] = [t]
    else:
        single_graph[h] += [t]
for i in range(ent_size):
    bdeg[i] = math.floor(math.log(i+1))

bin = int(max(bdeg))
rec_deg = np.zeros([ent_size,bin*depth])
for h in single_graph:
    set_len = len(single_graph[h])
    for i in single_graph[h]:
        rec_deg[h,int(bdeg[i])] += 1/set_len
for d in range(1,depth):
    
    for h in single_graph:
        set_len = len(single_graph[h])
        for i in single_graph[h]:
            rec_deg[h,d*bin:(d+1)*bin] += (rec_deg[i,(d-1)*bin:(d)*bin])/set_len


In [5]:
print(rec_deg.shape)

(38960, 30)


In [6]:
entity1, rel1, triples1 = load_triples('/content/GCN-Align/data/%s_en/'%lang + 'triples_1')
num_entity_1 = len(entity1)
num_rel_1 = len(rel1)

In [9]:
layer = SourceFileLoader("layer", "/content/RREA/CIKM/layer.py").load_module()

from layer import NR_GraphAttention
tf.keras.backend.clear_session()
radj = main_radj
radj1 = main_radj1
rel_matrix = main_rel_matrix
rel_matrix1 = main_rel_matrix1
node_size = adj_features.shape[0]
rel_size = rel_features.shape[1]
triple_size = len(adj_matrix)
batch_size = node_size
datas = []
# e stand for entity ,r stand for relation , E stand for encoder(e), c stand for character embedding , R stand for encoder(r)
run_name = 'eER'

def getMax(d):
    m = 0 
    max_key = -1;
    for k in d : 
        if d[k] > m :
            max_key = k
            m = d[k]
    return max_key

def get_match_relations(threshold = 50):
    rmap_predict = dict()
    train_pair_dic = dict()
    for i in range(len(train_pair)):
        train_pair_dic[train_pair[i][0]] = train_pair[i][1]
    triple_dic = dict()
    for h,t,r in radj:
        if r < (rel_size //2) :
            if (h,t) in triple_dic:
                triple_dic[(h,t)] += [r]
            else:
                triple_dic[(h,t)] = [r]
    for h,t,r in radj:
        if h in train_pair_dic and t in train_pair_dic and h != t and r < rel_size //2:
            z = np.zeros([rel_size])
            if (train_pair_dic[h],train_pair_dic[t]) in triple_dic:
                z[triple_dic[(train_pair_dic[h],train_pair_dic[t])]] = 1
                if r in rmap_predict :
                    rmap_predict[r] += z
                else:
                    rmap_predict[r] = z
    i = 0
    matched_dic = dict()
    for key in rmap_predict:
        v = np.max(rmap_predict[key])
        i += 1
        if(v>threshold):
            a = np.argmax(rmap_predict[key]) % (rel_size//2)
            matched_dic[key] = a
    #print(matched_dic);
    return matched_dic

def complete_graph(tpair,threshold=2,num = 10):
    
    new_train_pair = tpair.copy()
    for i in range(num):
        train_pair_dic1 = dict()
        for a,b in new_train_pair : 
            train_pair_dic1[a] = b
        selected_rmap = get_match_relations(threshold)

        si = dict()
        so = dict()

        for h,t,r in radj:
          if (t,r) not in si:
            si[(t,r)] = [h]
          else:
            si[(t,r)] += [h]

          if (h,r) not in so:
            so[(h,r)] = [t]
          else:
            so[(h,r)] += [t]

        lmap = dict()
        rmap = dict()
        tmap = dict()
        for h,t,r in radj:
            if h in train_pair_dic1 and r in selected_rmap :
                if (train_pair_dic1[h],selected_rmap[r]) in so:
                    for t1 in so[(train_pair_dic1[h],selected_rmap[r])]:
                        if t not in lmap:
                            lmap[t] = dict()
                        if t1 not in lmap[t]:
                            lmap[t][t1] = 1
                        else:
                            lmap[t][t1] += 1

            if t in train_pair_dic1 and r in selected_rmap  :
                if (train_pair_dic1[t],selected_rmap[r]) in si:
                    for h1 in si[(train_pair_dic1[t],selected_rmap[r])]:
                        if h not in rmap:
                            rmap[h] = dict()
                        if h1 not in rmap[h]:
                            rmap[h][h1] = 1
                        else:
                            rmap[h][h1] += 1
        for t in lmap:
            if t not in tmap:
                tmap[t] = dict()
            for key in lmap[t]:
                if key not in tmap[t]:
                    tmap[t][key] = lmap[t][key]
                else:
                    tmap[t][key] += lmap[t][key]

        for t in rmap:
            if t not in tmap:
                tmap[t] = dict()
            for key in rmap[t]:
                if key not in tmap[t]:
                    tmap[t][key] = rmap[t][key]
                else:
                    tmap[t][key] += rmap[t][key]
        l = []
        for a,b in pairs:
            if a not in train_pair_dic1 :
                if a in tmap:
                    train_pair_dic1[a] = getMax(tmap[a])
                    l.append([a,train_pair_dic1[a] ])
                elif a in lmap:
                    train_pair_dic1[a] = getMax(lmap[a])
                    l.append([a,train_pair_dic1[a] ])
                elif a in rmap:
                    train_pair_dic1[a] = getMax(rmap[a])
                    l.append([a,train_pair_dic1[a]])

        s = 0
        if (len(l) == 0):
            return train_pair_dic1;
        new_train_pair = np.concatenate([new_train_pair,np.array(l)],axis=0)
        for t in train_pair_dic1:
            if t not in train_pair_dic and pair_dic[t] == train_pair_dic1[t]:
                s += 1
        print(s/(len(pair_dic)-len(train_pair_dic)),len(train_pair_dic1)-len(train_pair_dic))
    return train_pair_dic1
def replce_matched_relations(threshold = 50):
    matched_dic = get_match_relations(threshold)
    for i in range(len(radj)) :
        h = radj[i,0]
        t = radj[i,1]
        r = radj[i,2]
        if r in matched_dic :
            radj[i,2] = matched_dic[r]
        elif (r - rel_size//2) in matched_dic:
            radj[i,2] = matched_dic[(r - rel_size//2)]   + (rel_size//2)

    for i in range(len(rel_matrix)) :
        h = rel_matrix[i,0]
        r = rel_matrix[i,1]
        if r in matched_dic :
            rel_matrix[i,1] = matched_dic[r]
        elif (r - rel_size//2) in matched_dic:
            rel_matrix[i,1] = matched_dic[(r - rel_size//2)]   + (rel_size//2)


class TokenEmbedding(keras.layers.Embedding):
    """Embedding layer with weights returned."""

    def compute_output_shape(self, input_shape):
        return self.input_dim, self.output_dim

    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, inputs):
        return self.embeddings
    
def get_embedding(add_rel_feature = False):
    inputs = [adj_matrix,r_index,r_val,rel_matrix,rel_matrix1,ent_matrix,radj,radj1,nadj]
    inputs = [np.expand_dims(item,axis=0) for item in inputs]
    if add_rel_feature == False:
        return get_emb.predict_on_batch(inputs)
    else:
        selected_rmap = get_match_relations(threshold)
        rselected_rmap = dict()
        i = 0
        for key in selected_rmap:
            rselected_rmap[key] = i
            rselected_rmap[selected_rmap[key]] = i
            i += 1
        depth = 1
        f = np.zeros([node_size,len(selected_rmap)*depth])
        for h,t,r in radj:
            if r in rselected_rmap:
                f[h,rselected_rmap[r]] += 1./degrees[h]
            
        for i in range(depth-1):
            for h,t,r in radj:
                f[h,(i+1)*depth:(i+2)*depth] += 1./degrees[h] * f[t,(i)*depth:(i+1)*depth]

        f1 = get_emb.predict_on_batch(inputs)
        f1_sum = f1.sum(axis=1)
        f1 = f1 / f1_sum[:, np.newaxis]
        f_sum = f.sum(axis=1)
        f = f / f_sum[:, np.newaxis]
        f2 = np.concatenate([f,f1],axis=1)
        return f

def test(wrank = None):
    vec = get_embedding()
    return  get_hits(vec,dev_pair,wrank=wrank)

def CSLS_test(thread_number = 16, csls=10,accurate = True):
    vec = get_embedding()
    trest_set_1 = [e1 for e1, e2 in dev_pair]
    trest_set_2 = [e2 for e1, e2 in dev_pair]
    Lvec = np.array([vec[e1] for e1, e2 in dev_pair])
    Rvec = np.array([vec[e2] for e1, e2 in dev_pair])

    Lvec = Lvec / np.linalg.norm(Lvec,axis=-1,keepdims=True)
    Rvec = Rvec / np.linalg.norm(Rvec,axis=-1,keepdims=True)

    # Lvec1 = np.array([rec_deg[e1] for e1, e2 in dev_pair])
    # Rvec1 = np.array([rec_deg[e2] for e1, e2 in dev_pair])

    # Lvec1 = 0.5 * Lvec1 / np.linalg.norm(Lvec1,axis=-1,keepdims=True)
    # Rvec1 = 0.5 * Rvec1 / np.linalg.norm(Rvec1,axis=-1,keepdims=True)

    # Lvec = np.concatenate([Lvec,Lvec1],axis=-1)
    # Rvec = np.concatenate([Rvec,Rvec1],axis=-1)

    # Lvec = Lvec / np.linalg.norm(Lvec,axis=-1,keepdims=True)
    # Rvec = Rvec / np.linalg.norm(Rvec,axis=-1,keepdims=True)

    lprec,ldata,_ = eval_alignment_by_sim_mat(Lvec, Rvec, [1, 5, 10], thread_number, csls=csls, accurate=accurate)
    results = np.zeros([len(dev_pair)])

    for i in range(len(dev_pair)):
        if((i,i) in lprec):
            results[i] = 1

    datas.append((results,ldata))
    save_datas()
    return None

def save_datas():
    with open('/content/RREA/'+lang+"_"+run_name+'.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([train_pair_degrees,dev_pair_degrees,datas], f)

def load_datas():
    with open('/content/RREA/'+lang+"_"+run_name+'.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
        mydata =pickle.load(f)
    return mydata;

def get_train_set(batch_size = batch_size):
    negative_ratio =  batch_size // len(train_pair) + 1
    train_set = np.reshape(np.repeat(np.expand_dims(train_pair,axis=0),axis=0,repeats=negative_ratio),newshape=(-1,2))
    np.random.shuffle(train_set); train_set = train_set[:batch_size]
    train_set = np.concatenate([train_set,np.random.randint(0,node_size,[train_set.shape[0],4])],axis = -1)
    return train_set

def get_train_set1(batch_size = batch_size):
    train_set = train_pair
    np.random.shuffle(train_set);
    train_set = np.concatenate([train_set,np.random.randint(0,node_size,train_set.shape)],axis = -1)
    return train_set

def get_trgat(node_size,rel_size,node_hidden,rel_hidden,triple_size,n_attn_heads = 2,dropout_rate = 0,gamma = 3,lr = 0.005,depth = 2):
    adj_input = Input(shape=(None,2))
    index_input = Input(shape=(None,2),dtype='int64')
    val_input = Input(shape = (None,))
    rel_adj = Input(shape=(None,2))
    rel_adj1 = Input(shape=(None,2))
    ent_adj = Input(shape=(None,2))
    radj = Input(shape=(None,3),dtype='int64')
    radj1 = Input(shape=(None,3),dtype='int64')
    nadj = Input(shape=(None,3))
    
    # ent_emb = TokenEmbedding(node_size,node_hidden,embeddings_initializer="LecunNormal",trainable = True)(val_input) 
    # rel_emb = TokenEmbedding(rel_size,node_hidden,embeddings_initializer="LecunNormal",trainable = True)(val_input)
    # ch_emb = TokenEmbedding(char_size,node_hidden,embeddings_initializer="LecunNormal",trainable = True)(val_input)

    ent_emb = TokenEmbedding(node_size,node_hidden,trainable = True)(val_input) 
    rel_emb = TokenEmbedding(rel_size,node_hidden,trainable = True)(val_input)
    ch_emb = TokenEmbedding(char_size,node_hidden,trainable = True)(val_input)


    # E = TokenEmbedding(node_hidden,node_hidden,trainable = True)(val_input)
    # R = TokenEmbedding(node_hidden,node_hidden,trainable = True)(val_input)
    def ch_avg(tensor,size):
        n_adj = tf.squeeze(tensor[0],axis = 0)
        adj = K.cast(n_adj[:,0:2],dtype = "int64")   
        adj = tf.SparseTensor(indices=adj, values=tf.ones_like(n_adj[:,2],dtype = 'float32') , dense_shape=(node_size,size)) 
        adj = tf.compat.v1.sparse_softmax(adj) 
        l_adj = tf.compat.v1.sparse_tensor_dense_matmul(adj,tensor[1])
        return l_adj

    def avg(tensor,size):
        adj = K.cast(K.squeeze(tensor[0],axis = 0),dtype = "int64")   
        adj = tf.SparseTensor(indices=adj, values=tf.ones_like(adj[:,0],dtype = 'float32'), dense_shape=(node_size,size)) 
        adj = tf.compat.v1.sparse_softmax(adj) 
        l_adj = tf.compat.v1.sparse_tensor_dense_matmul(adj,tensor[1])
        return l_adj

    name_feature = Lambda(ch_avg,arguments={'size':char_size})([nadj,ch_emb])
    # rel_feature1 = Lambda(avg,arguments={'size':rel_size//2})([rel_adj,rel_emb])
    # rel_feature2 = Lambda(avg,arguments={'size':rel_size//2})([rel_adj1,rel_emb])
    rel_feature = Lambda(avg,arguments={'size':rel_size})([rel_adj,rel_emb])
    rel_feature1= Lambda(avg,arguments={'size':rel_size})([rel_adj1,rel_emb])
    ent_feature = Lambda(avg,arguments={'size':node_size})([ent_adj,ent_emb])

    # rel_emb = K.concatenate([rel_emb1,rel_emb2])
    #rel_feature = K.concatenate([rel_feature1,rel_feature2])

    encoder = NR_GraphAttention(node_size,activation="relu",
                                       rel_size = rel_size,
                                       depth = 2,
                                       attn_heads=n_attn_heads,
                                       triple_size = triple_size,
                                       attn_heads_reduction='average',   
                                       dropout_rate=dropout_rate)
    encoder1 = NR_GraphAttention(node_size,activation="relu",
                                    rel_size = rel_size,
                                    depth = 2,
                                    attn_heads=n_attn_heads,
                                    triple_size = triple_size,
                                    attn_heads_reduction='average',   
                                    dropout_rate=dropout_rate)
    encoder2 = NR_GraphAttention(node_size,activation="relu",
                                       rel_size = rel_size,
                                       depth = 2,
                                       attn_heads=n_attn_heads,
                                       triple_size = triple_size,
                                       attn_heads_reduction='average',   
                                       dropout_rate=dropout_rate)
    opt1 = [rel_emb,adj_input,index_input,val_input,radj]
    opt2 = [rel_emb,adj_input,index_input,val_input,radj1]
    elements = []
    coefs = []
    for f_name in run_name:
        if f_name == 'e':
            elements += [ent_emb]
            coefs += [-0.5]
        elif f_name == 'E':
            elements += encoder([ent_feature]+opt1) 
            coefs += [1.,1.,1.]
        elif f_name == 'R':
            elements += encoder([rel_feature]+opt1) #+ encoder([rel_feature1]+opt2)
            coefs += [1.,1.,1.]
        elif f_name == 'q':
            elements += [rel_feature1]
            coefs +=  [-0.5]
        elif f_name == 'c':
            elements += [name_feature]
            coefs += [1.]

    #elements = [ent_emb] +   encoder([rel_feature]+opt1)+encoder([ent_feature]+opt1)
    #elements = [0.3 * ent_emb[:,:node_hidden//2]]+ el[0:3]+el[6:9]+[0.2 * ent_emb[:,node_hidden//2:]] + el[3:6]+el[9:12]
    num_el = elements.__len__()
    weight =  tf.Variable(coefs, trainable=False)
    weight = (num_el-2)* tf.math.softmax(weight)
    #weight =  tf.Variable(1 * [-0.2,1.0] + (num_el-2) * [.8], trainable=False)
    print(weight)
    elements = [weight[idx]*el for idx, el in enumerate(elements)]
    out_feature = Concatenate(-1)(elements)
    out_feature = Dropout(dropout_rate)(out_feature)
    
    alignment_input = Input(shape=(None,6),dtype = "int64") 
    #find = Lambda(lambda x:K.gather(reference=x[0],indices=K.cast(K.squeeze(x[1],axis=0), 'int32')))([out_feature,alignment_input])

    I = K.cast(K.squeeze(alignment_input,axis=0),dtype = "int64")
   
    A = K.sum(K.abs(K.gather(out_feature,I[:,0])-K.gather(out_feature,I[:,1])),axis=-1,keepdims=True)
    loss =  0.1 * ( K.relu(1 - K.sum(K.abs(K.gather(out_feature,I[:,0])))) +K.relu(1 - K.sum(K.abs(K.gather(out_feature,I[:,1])))))
    for i in range(2):
        B = K.sum(K.abs(K.gather(out_feature,I[:,0])-K.gather(out_feature,I[:,2+i])),axis=-1,keepdims=True)
        C = K.sum(K.abs(K.gather(out_feature,I[:,1])-K.gather(out_feature,I[:,2+i])),axis=-1,keepdims=True)
        loss += K.relu(gamma +A -B) + K.relu(gamma +A -C)
        loss += 0.1 * ( K.relu(1 - K.sum(K.abs(K.gather(out_feature,I[:,2+i])))))
    loss =  tf.compat.v1.reduce_sum(loss,keep_dims=True) / (batch_size)    
   
    inputs = [adj_input,index_input,val_input,rel_adj,rel_adj1,ent_adj,radj,radj1,nadj]
    train_model = keras.Model(inputs = inputs + [alignment_input],outputs = loss)
    
    train_model.compile(loss=lambda y_true,y_pred: y_pred,optimizer=tf.keras.optimizers.RMSprop(lr=lr))
    
    feature_model = keras.Model(inputs = inputs,outputs = out_feature)
    return train_model,feature_model

In [ ]:
model,get_emb = get_trgat(dropout_rate=0.3,node_size=node_size,rel_size=rel_size,n_attn_heads = 1,depth=2,gamma =3,node_hidden=100,rel_hidden = 100,triple_size = triple_size)
#model.summary(); 
initial_weights = model.get_weights()

In [ ]:
a = get_embedding(True)

In [ ]:
train_pair = train_pair_main
radj = main_radj
radj1 = main_radj1
rel_matrix = main_rel_matrix
rel_matrix1 = main_rel_matrix1
tf.keras.backend.clear_session()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True  
sess = tf.compat.v1.Session(config=config) 

rest_set_1 = [e1 for e1, e2 in dev_pair]
rest_set_2 = [e2 for e1, e2 in dev_pair]
np.random.shuffle(rest_set_1)
np.random.shuffle(rest_set_2)

epoch = 500
for turn in range(20):
    print("iteration %d start."%turn)
    for i in trange(epoch):
        cur_epoch = i
        train_set = get_train_set()
        
        inputs = [adj_matrix,r_index,r_val,rel_matrix,rel_matrix1,ent_matrix,radj,radj1,nadj,train_set]
        inputs = [np.expand_dims(item,axis=0) for item in inputs]
        model.train_on_batch(inputs,np.zeros((1,1)))
        #CSLS_test()
        if i%100 == 99 and i+1 != epoch:
            
            CSLS_test()     
    
    if turn == 3:
        epoch = 100

    CSLS_test()       
    new_pair1 = []
    new_pair = []
    
    vec = get_embedding()
    Lvec = np.array([vec[e] for e in rest_set_1])
    Rvec = np.array([vec[e] for e in rest_set_2])
    Lvec = Lvec / np.linalg.norm(Lvec,axis=-1,keepdims=True)
    Rvec = Rvec / np.linalg.norm(Rvec,axis=-1,keepdims=True)
    A,_,_ = eval_alignment_by_sim_mat(Lvec, Rvec, [1, 5, 10], 16,10,True,False)
    B,_,_ = eval_alignment_by_sim_mat(Rvec, Lvec,[1, 5, 10], 16,10,True,False)
    A = sorted(list(A)); B = sorted(list(B))
    for a,b in A:
        if b>= 0 and  B[b][1] == a:
            new_pair1.append([rest_set_1[a],rest_set_2[b]])

    # _,_,A = eval_alignment_by_sim_mat(Lvec, Rvec, [1, 5, 10], 16,10,True,False)
    # _,_,B = eval_alignment_by_sim_mat(Rvec, Lvec,[1, 5, 10], 16,10,True,False)
    # A = A[0]
    # B = B[0]

    # for i in range(len(A)):
    #     if i == B[A[i,0],0]:
    #         has_other_pair = 0
    #         for _i in range(1,10):
    #             if(_i in B[A[i,_i],:10]):
    #                 has_other_pair = 1
    #                 break;

    #         if has_other_pair == 0:
    #               #print([rest_set_1[i],rest_set_2[A[i,j]]])
    #               new_pair.append([rest_set_1[i],rest_set_2[A[i,0]]])
    new_pair_dic = complete_graph(train_pair,2,10) 
    for a,b in new_pair1:
        if (a in new_pair_dic and new_pair_dic[a] == b):
            new_pair.append([a,b])

    train_pair = np.concatenate([train_pair,np.array(new_pair)],axis = 0)
    s = 0
    s1 = 0
    for a,b in new_pair :
        if pair_dic[a] == b:
            s1 += 1
    for a,b in train_pair:
        if a not in train_pair_dic and pair_dic[a] == b:
            s += 1

    print("generate new semi-pairs: " + str(len(new_pair)) + ".  num of correct new pairs" + str(s1))
    print("total new pairs: " + str(len(train_pair) - len(train_pair_dic)) + ".  num of correct new pairs " + str(s))

    for e1,e2 in new_pair:
        if e1 in rest_set_1:
            rest_set_1.remove(e1) 
        
    for e1,e2 in new_pair:
        if e2 in rest_set_2:
            rest_set_2.remove(e2)
            
    with open('/content/RREA/'+lang+"_"+run_name+'_train_pair'+str(turn)+'.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(train_pair, f)

    complete_graph(train_pair,2,10)
    #match_relations()
for turn in range(5):
    files.download('/content/RREA/'+lang+"_"+run_name+'_train_pair'+str(turn)+'.pkl');
files.download('/content/RREA/'+lang+"_"+run_name+'.pkl') 

iteration 0 start.


 20%|██        | 100/500 [03:07<59:22,  8.91s/it]

accurate results: hits@[1, 5, 10] = [64.8  86.71 91.89], mr = 11.888, mrr = 0.745, time = 23.174 s 


 40%|████      | 200/500 [06:04<42:33,  8.51s/it]

accurate results: hits@[1, 5, 10] = [68.78 88.69 92.68], mr = 13.402, mrr = 0.775, time = 22.431 s 


...

100%|██████████| 100/100 [02:33<00:00,  1.53s/it]


accurate results: hits@[1, 5, 10] = [79.11 91.68 94.66], mr = 13.738, mrr = 0.847, time = 22.488 s 
0.6244761904761905 9408
0.6251428571428571 9557
0.6251428571428571 9572
0.6251428571428571 9576
0.6251428571428571 9578
generate new semi-pairs: 15.  num of correct new pairs12
total new pairs: 6461.  num of correct new pairs 6340
0.6250476190476191 9411
0.6255238095238095 9557
0.6255238095238095 9572
0.6255238095238095 9576
0.6255238095238095 9578
iteration 8 start.


100%|██████████| 100/100 [02:27<00:00,  1.47s/it]


accurate results: hits@[1, 5, 10] = [79.22 91.91 94.61], mr = 14.086, mrr = 0.847, time = 22.684 s 
0.6250476190476191 9411
0.6255238095238095 9557
0.6255238095238095 9572
0.6255238095238095 9576
0.6255238095238095 9578
generate new semi-pairs: 7.  num of correct new pairs7
total new pairs: 6468.  num of correct new pairs 6347
0.6251428571428571 9411
0.6256190476190476 9557
0.6256190476190476 9572
0.6256190476190476 9576
0.6256190476190476 9578
iteration 9 start.


 33%|███▎      | 33/100 [00:48<01:39,  1.48s/it]

In [ ]:
new_pair_dic = complete_graph(train_pair,2,10)

0.31847619047619047 6156
0.4046666666666667 8646
0.41133333333333333 8848
0.4119047619047619 8872
0.4119047619047619 8874
0.412 8875
0.412 8876


In [ ]:
    for a,b in new_pair1:
        if (a in new_pair_dic and new_pair_dic[a] == b):
            new_pair.append([a,b])

In [ ]:
len(new_pair)

3850

In [ ]:
A,_,_ = eval_alignment_by_sim_mat(Lvec, Rvec, [1, 5, 10], 16,10,True,False)
B,_,_ = eval_alignment_by_sim_mat(Rvec, Lvec,[1, 5, 10], 16,10,True,False)
A = sorted(list(A)); B = sorted(list(B))
for a,b in A:
    if b>= 0 and  B[b][1] == a:
        new_pair.append([rest_set_1[a],rest_set_2[b]])

In [ ]:
    s = 0
    s1 = 0
    for a,b in new_pair :
        if pair_dic[a] == b:
            s1 += 1

In [ ]:
3808/3850

0.9890909090909091

In [ ]:
CSLS_test()

accurate results: hits@[1, 5, 10] = [64.18 86.42 91.21], mr = 11.415, mrr = 0.739, time = 26.910 s 


In [ ]:
    print("generate new semi-pairs: " + str(len(new_pair)) + ".  num of correct new pairs" + str(s1))
    print("Current train pairs : " + str(len(train_pair)) + ".  num of correct train pairs" + str(s))

generate new semi-pairs: 5371.  num of correct new pairs4100
Current train pairs : 9871.  num of correct train pairs4100
